# Step 14, 같은 변수 반복 사용

현재 문제는 같은 변수를 반복해서 사용할  경우 의도대로 동작하지 않을 수 있다는 문제  
y = add(x,x) 계산을 예

In [8]:
import numpy as np
from step13 import add

In [9]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = [self.creator]                      # 함수들을 담는 리스트 
        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output.grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                x.grad = gx                                 # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장

                if x.creator is not None:
                    funcs.append(x.creator)

x = Variable(np.array(3.0))
y = add(x,x)
print('y',y.data)

y.backward()
print('x.grad',x.grad)

y 6.0
x.grad 1.0


x = 3.0  
y는 6으로 잘 나옴  
그러나 y = x0 + x1 => y = x + x => y = 2x 이므로  
y' = 2이여야 한다.

## 14.1 문제의 원인

class Variable 에서 

In [ ]:
class Variable:
    # ... 생략 ....
        funcs = [self.creator]                      
        while funcs:
            f = funcs.pop()                         
            gys = [output.grad for output in f.outputs]     
            gxs = f.backward(*gys)                          
            if not isinstance(gxs, tuple):                  
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):               
                x.grad = gx                               # 여기가 문제 

                if x.creator is not None:
                    funcs.append(x.creator)

출력 똑에서 전해지는 미분값을 그대로 대입하고 있다.  
따라서 같은 변수를 반복해서 사용하면 전파되는 미분값이 덮어 써진다.

x의 미분은 1 + 1 = 2가 되어야 올바른 결과이다.  
즉, 전파되는 미분값의 '합'을 구해야 한다. 그러나 지금 구현에서는 그냥 덮어쓰고 있다.

## 14.2 해결책

In [ ]:
class Variable:
    
# ...... 생략 .........

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:                          # x.grad가 기존에 없으면, 처음 받는거면, 이제 하나이면
                    x.grad = gx
                else:                                       # x.grad가 기존 값을 가지고 있으면 거기에 덧붙여서 더함
                    x.grad = x.grad + gx                                 

                if x.creator is not None:
                    funcs.append(x.creator)


최종 코드

In [18]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)



x = Variable(np.array(3.0))
y = add(x,x)
print('y',y.data)

y.backward()
print('x.grad',x.grad)

y 6.0
x.grad 2.0


In [19]:
x = Variable(np.array(3.0))
y = add(add(x,x),x)         # x + x + x
y.backward()
print(x.grad)

3.0


## 14.3 미분값 재설정 

새로운 문제 발생 : 같은 변수를 사용하여 '다른'계산을 할 경우 계산이 꼬이는 문제 

In [21]:
# 첫번째 계산 
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

# 두번째 계산(같은 x를 사용하여 다른 계산을 수행)
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
5.0


위 코드는 서로 다른 두 가지 미분 계산을 수행하였다.  
그러면서 메모리를 절약하고자 Variable 인스턴스인 x를 재사용했다.  
그 결과 두 번째 x의 미분값에 첫번째 미분값이 더해지고, 5.0이라는 잘못된 값을 돌려준다.(3+2=5) (결과는 3.0이 되어야한다.)

이 문제를 해결하기 위해 Variable 클래스에 미분값을 초기화하는 cleargrad 메서드를 추가한다.

In [22]:
class Variable : 
    # .... 생략 ....
    def cleargrad(self):
        self.grad = None

cleargrad는 미분값을 초기화하는 메서드로, 단순히 self.grad에 None을 대입한다.  
이 메서드를 사용하면 여러가지 미분을 연달아 계산할 때 똑같은 변수를 재사용할 수 있다.

In [38]:
import numpy as np
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)


In [39]:
# 첫번째 계산 
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

# 두번째 계산(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad()           # 미분값을 초기화, 두번때 x.backward()를 호출하기 전에 x.cleargrad()를 호출하면 변수에 누적된 미분값이 초기화 
y = add(add(x,x),x)
y.backward()
print(x.grad)

2.0
3.0
